<a href="https://colab.research.google.com/github/JackiApligian/ML-ReCell/blob/main/Apligian_ReCell_FullCode_Project_LearnerNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Learning - Foundations Project: ReCell 

## Problem Statement

### Business Context

Buying and selling used phones and tablets used to be something that happened on a handful of online marketplace sites. But the used and refurbished device market has grown considerably over the past decade, and a new IDC (International Data Corporation) forecast predicts that the used phone market would be worth \\$52.7bn by 2023 with a compound annual growth rate (CAGR) of 13.6% from 2018 to 2023. This growth can be attributed to an uptick in demand for used phones and tablets that offer considerable savings compared with new models.

Refurbished and used devices continue to provide cost-effective alternatives to both consumers and businesses that are looking to save money when purchasing one. There are plenty of other benefits associated with the used device market. Used and refurbished devices can be sold with warranties and can also be insured with proof of purchase. Third-party vendors/platforms, such as Verizon, Amazon, etc., provide attractive offers to customers for refurbished devices. Maximizing the longevity of devices through second-hand trade also reduces their environmental impact and helps in recycling and reducing waste. The impact of the COVID-19 outbreak may further boost this segment as consumers cut back on discretionary spending and buy phones and tablets only for immediate needs.

 
### Objective

The rising potential of this comparatively under-the-radar market fuels the need for an ML-based solution to develop a dynamic pricing strategy for used and refurbished devices. ReCell, a startup aiming to tap the potential in this market, has hired you as a data scientist. They want you to analyze the data provided and build a linear regression model to predict the price of a used phone/tablet and identify factors that significantly influence it.

 
### Data Description

The data contains the different attributes of used/refurbished phones and tablets. The data was collected in the year 2021. The detailed data dictionary is given below.


- brand_name: Name of manufacturing brand
- os: OS on which the device runs
- screen_size: Size of the screen in cm
- 4g: Whether 4G is available or not
- 5g: Whether 5G is available or not
- main_camera_mp: Resolution of the rear camera in megapixels
- selfie_camera_mp: Resolution of the front camera in megapixels
- int_memory: Amount of internal memory (ROM) in GB
- ram: Amount of RAM in GB
- battery: Energy capacity of the device battery in mAh
- weight: Weight of the device in grams
- release_year: Year when the device model was released
- days_used: Number of days the used/refurbished device has been used
- normalized_new_price: Normalized price of a new device of the same model in euros
- normalized_used_price: Normalized price of the used/refurbished device in euros

## Importing necessary libraries

In [ ]:
#Importing all necessary libraries for data manipulation and visalization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

#Importing libraries for data analysis and stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Loading the dataset

In [ ]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')
#loading dataset from drive
data=pd.read_csv('/content/drive/My Drive/used_device_data.csv')

## Data Overview

- Observations
- Sanity checks

In [ ]:
#checking the first few rows of the dataset
data.head()

In [ ]:
#Checking the last few rows of the dataset
data.tail()

In [ ]:
#Checking the shape of the dataset
data.shape

**There are 3454 rows of data and 15 columns**

In [ ]:
#Checking the dtypes of the data
data.info()

**There are missing values in this data frame that will need to be addressed.**

In [ ]:
  #Checking the value counts for the release year
  data.value_counts('release_year')

**Multiple years are included in this data frame. I will need to decide how to format as we look further into the data.**

In [ ]:
#Looking at the statistical summary of the data
data.describe(include='all').T

**There are 34 unique brands in this data, with 4 unique operating systems. The average screen size is 13.7, there are more phones with 4g than 5g capabilities, the average release year is 2015 and average days used is 674.86 days.**

In [ ]:
#Checking for duplicated data
data.duplicated().sum()

**There are no duplicated values.**

In [ ]:
#Checking the df for missing values
data.isnull().sum()

**There are quite a few missing value with the majority in the main_camera_mp column.**

In [ ]:
#making a copy of the original dataframe so that it remains unchanged
df=data.copy()

## Exploratory Data Analysis (EDA)

- EDA is an important part of any project involving data.
- It is important to investigate and understand the data better before building a model with it.
- A few questions have been mentioned below which will help you approach the analysis in the right manner and generate insights from the data.
- A thorough analysis of the data, in addition to the questions mentioned below, should be done.

**Questions**:

1. What does the distribution of normalized used device prices look like?
2. What percentage of the used device market is dominated by Android devices?
3. The amount of RAM is important for the smooth functioning of a device. How does the amount of RAM vary with the brand?
4. A large battery often increases a device's weight, making it feel uncomfortable in the hands. How does the weight vary for phones and tablets offering large batteries (more than 4500 mAh)?
5. Bigger screens are desirable for entertainment purposes as they offer a better viewing experience. How many phones and tablets are available across different brands with a screen size larger than 6 inches?
6. A lot of devices nowadays offer great selfie cameras, allowing us to capture our favorite moments with loved ones. What is the distribution of devices offering greater than 8MP selfie cameras across brands?
7. Which attributes are highly correlated with the normalized price of a used device?

In [ ]:
#funtion to create EDA with boxplot and histogram plots on the same scale
def histogram_boxplot(data,feature,figsize=(15,10),kde=False, bins=None):
  """
  Boxplot and histogram combined

  data:dataframe
  feature:dataframe column
  figsize: size of the figure (default(15,10))
  kde: whether to show the density curve (default False)
  bins: number of bins for histogram (default None)
  """
  #creating the 2 subplots
  f2, (ax_box2, ax_hist2) = plt.subplots(nrows=2,#number of rows in the subplot grid = 2
      sharex=True,#x-axis to be shared on all subplots
      gridspec_kw={'height_ratios': (.25,.75)},
      figsize=figsize, )
 
  #boxplot will be created with a triangle showing the mean value of a column
  sns.boxplot (data=data, x=feature, ax=ax_box2, showmeans=True, color= 'violet')
  
  # for histograms
  sns.histplot(data=data,x=feature,kde=kde,ax=ax_hist2, bins=bins)if bins else sns.histplot(data=data, x=feature, kde=kde, ax=ax_hist2)
    
     #add means to histograms
  ax_hist2.axvline(data[feature].mean(),color='green', linestyle='--')
    
    #adding median to histograms
  ax_hist2.axvline(data[feature].median(), color='black', linestyle='-')


In [ ]:
# function to create labeled barplots


def labeled_barplot(data, feature, perc=False, n=None):
    """
    Barplot with percentage at the top

    data: dataframe
    feature: dataframe column
    perc: whether to display percentages instead of count (default is False)
    n: displays the top n category levels (default is None, i.e., display all levels)
    """
# length of the column
    total = len(data[feature])  
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count + 2, 6))
    else:
        plt.figure(figsize=(n + 2, 6))

    plt.xticks(rotation=90, fontsize=15)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Paired",
        order=data[feature].value_counts().index[:n],
    )
# percentage of each class of the category
    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
                100 * p.get_height() / total
            )  
        else:
            label = p.get_height()  # count of each level of the category
# width of the plot
        x = p.get_x() + p.get_width() / 2  
# height of the plot
        y = p.get_height()  
 # annotate the percentage
        ax.annotate(
            label,
            (x, y),
            ha="center",
            va="center",
            size=12,
            xytext=(0, 5),
            textcoords="offset points",
        ) 
# show the plot
    plt.show()  

**UNIVARIATE ANALYSIS**

In [ ]:
#distribution of normalized used price
histogram_boxplot(data, "normalized_used_price")

1.What does the distribution of normalized used device prices look like?
**The normalized used price has a slight left skew but with a normal distribution curve. The average price for a used phone is slightly less than 4.5 with 250 phones at that price. The boxplot shows that there are far more ouliers in the lower price range than the upper. The mean and median are close to the same amount.**

In [ ]:
#distribution of normalized new price
histogram_boxplot(data,"normalized_new_price")

**The normalized new price data follows the pattern of normalized distribution but has far more dips in the pattern compared to the used price distribution. The average price is around 5.2 but the majority of phones are priced at 5. The mean and median are close to the same amount.** 

In [ ]:
#distribution of screen size
histogram_boxplot(data, "screen_size")

**Screen size appears to have a relatively flat distribution except for the median amount that is around 13. The median and mean are slightly seperated.**

In [ ]:
#distribution of main camera mp
histogram_boxplot(data,'main_camera_mp')

**There is a right skewed distribution that is not normalized. The mean and median are slightly seperated and the mode is removed from them both.**

In [ ]:
#distribution of selfie camera mp
histogram_boxplot(data,'selfie_camera_mp')

**The selfie camera mp data is right skewed with many outliers to the higher end of the megapixel count. The median and mean are seperated and the data is not normalized.**

In [ ]:
#distribution of internal memory
histogram_boxplot(data, 'int_memory')

**The internal memory data is heavily skewed to the right with the median and mean seperated and the mode less than the median and mean. There is a huge outlier amount to the higher end of the data for this variable.**

In [ ]:
#distribution of ram
histogram_boxplot(data, 'ram')

**The ram data is right skewed with a relatively flat distribution except for the median and mean values of 4 ram. There is a large difference in the number of phones with 4 ram than any other amount. There are over 2500 phones in the data with 4 ram while the other ram amounts do not have more than 250 phones in the data.**

In [ ]:
#distribution of weight
histogram_boxplot(data,'weight')

**The weight of the phones in the data is heavily skewed to the right. The information of phones less than 200 is normalized followed by many phones weighing more than 200 in a relatively flat distribution pattern. The median and mode are seperated.**

In [ ]:
#distribution of the battery
histogram_boxplot(data, 'battery')

**The battery data has a right skew with the meidan and mean lining up with the mode. The majority of battery are at the 2000, 3000 and 4000 values.**

In [ ]:
#distribution of days used
histogram_boxplot(data, "days_used")

**The days used has a left skew with the mode being less than the seperated median and mean amounts.**

In [ ]:
data.value_counts('brand_name')

In [ ]:
#brand name distribution
labeled_barplot(data,'brand_name', perc=True, n=33)

**Most of the phones in the data frame are in the "Others" catagory with the next highest number being Samsung phones at 9.9%. The brand with the smallest number of phones in the data is Google at .4%.**

In [ ]:
#distribution of OS
labeled_barplot(data,"os", perc=True, n=10)

2.What percentage of the used device market is dominated by Android devices?
**The majority of phones have the Android operating system at 93.1%. The iOS has the least amount at 1.0%.**

In [ ]:
labeled_barplot(data,'4g', perc=True, n=2 )

**67.6% of the phones in the dataframe have 4g capability and 32.4% do not.**

In [ ]:
#distribution of 5g
labeled_barplot(data,'5g', perc=True, n=2)

**95.6% of the phones in the dataframe do not have 5g capabilities while 4.4% do.**

In [ ]:
labeled_barplot(data,'release_year', perc=True, n=10)

**The majority of phones in this dataframe were released in 2014 at 18.6% and the least amount were from 2020 at 8%.**

**BIVARIATE ANALYSIS**

In [ ]:
#Checking for coorelations in the data
cols_list = df.select_dtypes(include=np.number).columns.tolist()
#Because release year is a time variable, it needs to be dropped
cols_list.remove("release_year")

plt.figure(figsize=(20, 15))
sns.heatmap(
    df[cols_list].corr(), annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="Spectral"
)
plt.show()

**The battery and screen size as well as weight and screen size have high coorelation values. Weight and battery are also coorelated. There is little coorelation between days used and any of the other variables.** 
7.Which attributes are highly correlated with the normalized price of a used device?**The highest coorelation is with the normalized new price values. The features that seem to be the highest correlation to normalized used price would be the screen size, selfie camera mp, and battery.**

In [ ]:
#comparing the amount of RAM to the different devices
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x="brand_name", y="ram")
plt.xticks(rotation=90)
plt.show()

3. The amount of RAM is important for the smooth functioning of a device. How does the amount of RAM vary with the brand? **There is a great deal of variance across devices. The device that appears to have the most consistly high levels of RAM would be OnePlus. The device that appears to have the most consistly low levels of RAM would be Celkon. There are outliers for high RAM for Huawei, Micromax, Oppo, Samsung and Xiaomi.**

In [ ]:
#comparing battery size across phones
#breaking the heavier phones into a new df
df_large_batt = df[df.battery > 4500]
df_large_batt.shape



**The new dataframe has 341 rows and 15 columns so there were 341 phones in our original data that weight over 4500.**

In [ ]:
#barplot comparing brand name to weight
plt.figure(figsize=(10, 5))
sns.boxplot(data = df_large_batt, x='brand_name', y='weight')
plt.xticks(rotation=90)
plt.show()

4.A large battery often increases a device's weight, making it feel uncomfortable in the hands. How does the weight vary for phones and tablets offering large batteries (more than 4500 mAh)? **There is a great deal of variance across the phones in weight. The heaviest is an outlier in the Others catagory followed by Samsung. The Lenovo has the highest mean weight of all of the phones.**

In [ ]:
#comparing screen size across phones
#breaking the phones with a larger screen size into a seperate df
df_large_screen = df[df.screen_size > 6 * 2.54]
df_large_screen.shape

**There are now 1099 rows and 15 columns which means there were 1099 phones with larger screens in the original data frame.**

In [ ]:
#barplot to compare screen size to brand name
labeled_barplot(df_large_screen, 'brand_name')


5.Bigger screens are desirable for entertainment purposes as they offer a better viewing experience. How many phones and tablets are available across different brands with a screen size larger than 6 inches? **The majority of larger screens are from Huawei at 149 with Samsung close behind at 119.**

In [ ]:
#comparing selfie cameras across phones
#breaking the phones with a greater than 8mp selfie camera into a seperate df
df_selfie_camera = df[df.selfie_camera_mp > 8]
df_selfie_camera.shape

**There are 655 rows and 15 columns in this new dataframe so there are 655 phones with a selfie camera with greater than 8 megapixels.**

In [ ]:
#barplot to compare selfie camera to brand
labeled_barplot(df_selfie_camera, 'brand_name')

6.A lot of devices nowadays offer great selfie cameras, allowing us to capture our favorite moments with loved ones. What is the distribution of devices offering greater than 8MP selfie cameras across brands? **The most phones with over 8 megapixels selfie cameras are from Huawei, Vivo and Xiaomi. Samsung is also high on the list.**

In [ ]:
#doing the same analysis with the rear camera
#breaking the phones with a greater than 14mp rear camera into a seperate df
df_rear_camera = df[df.main_camera_mp > 14]
df_rear_camera.shape

**There are 261 rows and 15 columns in this dataframe. So there are 261 phones in the original dataset that have over 14 mp in the rear camera.**

In [ ]:
#comparing rear camera mp across brands
labeled_barplot(df_rear_camera, 'brand_name')

**Samsung has the most phones with over 14 megapixal rear cameras at 44, followed by Sony at 37 and Others at 36.**

In [ ]:
#comparing price of the used phones to year released
plt.figure(figsize=(15, 5))
sns.lineplot(data=data, y='normalized_used_price', x='release_year') 
plt.show()

**The price goes up as the phone release year increases. So the newer the phone the more it costs.**

In [ ]:
#Comparing price to 4g and 5g capabilities
plt.figure(figsize=(10, 4))

plt.subplot(121)
sns.boxplot(data=df, x="4g", y="normalized_used_price")

plt.subplot(122)
sns.boxplot(data=df, x="5g", y="normalized_used_price")

plt.show()

**The average price of the phones with 4g OR 5g capabilities are higher than those without. The price for the phones with 4g and without 5g are very similar which makes me question if they are the same data points.**

## Data Preprocessing

- Missing value treatment
- Feature engineering (if needed)
- Outlier detection and treatment (if needed)
- Preparing data for modeling
- Any other preprocessing steps (if needed)

In [ ]:
#checking for missing values
df.isnull().sum()

In [ ]:
#fixing the missing values in internal memory and ram by using the median for the release year and brand name to fill the value.
cols_impute = [
    "main_camera_mp",
    "selfie_camera_mp",
    "int_memory",
    "ram",
    "battery",
    "weight",
]

for col in cols_impute:
    df[col] = df[col].fillna(
        value=df.groupby(['release_year', 'brand_name'])[col].transform("median")
    )

# checking for missing values
df.isnull().sum()

**The internal memory and RAM missing values have been replaced by the release year and brand name median for that data point.**

In [ ]:
#fixing the missing values in the main camera mp, selfie camera mp, battery and weight columns by using the average of the brand names for those values.
cols_impute = [
    "main_camera_mp",
    "selfie_camera_mp",
    "battery",
    "weight",
]

for col in cols_impute:
    df[col] = df[col].fillna(
        value=df.groupby(['brand_name'])[col].transform("median")
    ) 

# checking for missing values
df.isnull().sum()

**These changes took care of all but 10 null values in the main camera mp column.**


In [ ]:
#changing remainging missing values in the main camera mp column to the median of the entire main camera mp column
df["main_camera_mp"] = df["main_camera_mp"].fillna(df["main_camera_mp"].median())
# checking for missing values
df.isnull().sum()

**All null or missing values have been replaced.**

In [ ]:
#Since release year is a time variable, it should be changed so that it can be analyzed. Changing from release year to years since release using 2021 as the start year.
#dropping the release year column.
df["years_since_release"] = 2021 - df["release_year"]
df.drop("release_year", axis=1, inplace=True)
df["years_since_release"].describe()

In [ ]:
#checking for outliers
num_cols = df.select_dtypes(include=np.number).columns.tolist()

plt.figure(figsize=(20, 20))

for i, variable in enumerate(num_cols):
    plt.subplot(4, 3, i + 1)
    sns.boxplot(data=df, x=variable)
    plt.tight_layout(pad=2)

plt.show()

**There are numerous outliers in this data set but no treatment is needed because they are accurate values.**

## EDA

- It is a good idea to explore the data once again after manipulating it.

In [ ]:
#distribution of main camera mp after changes
histogram_boxplot(data,'main_camera_mp')

**Visualization shows little change from previous plot.**

In [ ]:
#distribution of selfie camera mp after changes
histogram_boxplot(data,'selfie_camera_mp')

**Visualization shows little change from previous plot.**:

In [ ]:
#distribution of the battery after changes
histogram_boxplot(data, 'battery')

**Visualization shows little change from previous plot.**

In [ ]:
#distribution of weight after changes
histogram_boxplot(data,'weight')

**Visualization shows little change from previous plot.**

In [ ]:
#barplot comparing brand name to weight
plt.figure(figsize=(10, 5))
sns.boxplot(data = df_large_batt, x='brand_name', y='weight')
plt.xticks(rotation=90)
plt.show()

**Visualization shows little change from previous plot.**

**These finding with the reanalysed EDA show that the changes made to the missing values did not affect the overall data in a meaningful way. This is good news.**

## Model Building - Linear Regression

In [ ]:
#defining independent and dependent variables
# independent variables
X = df.drop(["normalized_used_price"], axis=1)
# dependent variable
y = df["normalized_used_price"]

**For this model, the Normalized Used Price will be the dependent variable.**

In [ ]:
print(X.head())
print()
print(y.head())

In [ ]:
# adding intercept to data
X = sm.add_constant(X)

In [ ]:
#creating dummies for the object dtypes for calculations
X = pd.get_dummies(X,
    columns=X.select_dtypes(include=["object", "category"]).columns.tolist(),
    drop_first=True)
X.head()

In [ ]:
#spliting the data into 70% training and 30% testing  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [ ]:
print(X_train.head())

In [ ]:
print(X_train.shape[0])

**There are 2417 rows in the training data**

In [ ]:
print(X_test.head())

In [ ]:
print(X_test.shape[0])

**There are 1037 rows in the testing data**

In [ ]:
#fitting the OLS model on the training data
olsmodel1 = sm.OLS(y_train,X_train)
olsres= olsmodel1.fit()

In [ ]:
#printing the OLS summary
print(olsres.summary())

**The adjusted R-Squared value is .842 which lets us know that this model reflect an 84% fit to the data which suggests it is a good model.**

## Model Performance Check

**In order to accept our model as a good fit, we need more information. We do this by creating funtions that calculate the MAPE and adjust R2.**

In [ ]:
# creating a function to find adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# creating a function to find MAPE
def mape_score(targets, predictions):
    return np.mean(np.abs(targets - predictions) / targets) * 100


# creating a function to find the different metrics to check performance of the regression model
def model_performance_regression(model, predictors, target):
    """
    Creating a function to find the different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

# creatin a model to predict using the independent variables
    pred = model.predict(predictors)
# to compute R-squared
    r2 = r2_score(target, pred)  
# to compute adjusted R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  
# to compute RMSE    
    rmse = np.sqrt(mean_squared_error(target, pred)) 
# to compute MAE
    mae = mean_absolute_error(target, pred)  
# to compute MAPE
    mape = mape_score(target, pred)  

# creating a dataframe of the metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf

In [ ]:
#checking the model performance on training set with 70% of the data
print("Training Performance\n")
olsres_train_perf = model_performance_regression(olsres, X_train, y_train)
olsres_train_perf

In [ ]:
#checking the model performance on testing set with 30% of the data
print("Test Performance\n")
olsres_test_perf = model_performance_regression(olsres, X_test, y_test)
olsres_test_perf

**The Adjust R-squared for the training data is 84% and for the test data is 83%. This information lets us know that the model is not underfit. The training and test RSME and MAE are comparable so this further suggests that the model is not underfit. MAPE of 4.5 on the test data means that this model can predict within 4.5% of the normalized used price of a device.**

## Checking Linear Regression Assumptions

- In order to make statistical inferences from a linear regression model, it is important to ensure that the assumptions of linear regression are satisfied.

In [ ]:
#Testing for multicollinearity using VIF.
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_series1 = pd.Series(
    [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])],
    index=X_train.columns,
)
print("VIF values: \n\n{}\n".format(vif_series1))

**When interpreting VIF:**

-If VIF is 1 then there is no correlation between the predictor and the remaining predictor variables.
-If VIF exceeds 5 or is close to exceeding 5, we say there is moderate multicollinearity.
-If VIF is 10 or exceeding 10, it shows signs of high multicollinearity.

**With the above knowledge we see that there are quite a few variables that show a moderate to high multi-collinearity. These values need to be treated.**

In [ ]:
#Testing for multicollinearity using VIF.
from statsmodels.stats.outliers_influence import variance_inflation_factor

# we will define a function to check VIF
def checking_vif(predictors):
    vif = pd.DataFrame()
    vif["feature"] = predictors.columns

    # calculating VIF for each feature
    vif["VIF"] = [
        variance_inflation_factor(predictors.values, i)
        for i in range(len(predictors.columns))
    ]
    return vif

In [ ]:
def treating_multicollinearity(predictors, target, high_vif_columns):
    """
    Checking the effect of dropping the columns showing high multicollinearity
    on model performance (adj. R-squared and RMSE)

    predictors: independent variables
    target: dependent variable
    high_vif_columns: columns having high VIF
    """
# creating empty lists to store adj. R-squared and RMSE values
    adj_r2 = []
    rmse = []

# building the ols models by dropping one of the high VIF columns at a time
# storeing the adjusted R-squared and RMSE in the lists defined previously
    for cols in high_vif_columns:
# defining the new training set
        train = predictors.loc[:, ~predictors.columns.str.startswith(cols)]

# creating the model
        olsmodel = sm.OLS(target, train).fit()

# adding adj. R-squared and RMSE to the lists
        adj_r2.append(olsmodel.rsquared_adj)
        rmse.append(np.sqrt(olsmodel.mse_resid))
# creating a dataframe for the results
    temp = pd.DataFrame({"col": high_vif_columns,
            "Adj. R-squared after_dropping col": adj_r2,
            "RMSE after dropping col": rmse,}).sort_values(by="Adj. R-squared after_dropping col", ascending=False)
    temp.reset_index(drop=True, inplace=True)
    return temp

In [ ]:
#Testing for multicollinearity using VIF.
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_series1 = pd.Series(
    [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])],
    index=X_train.columns,
)
print("VIF values: \n\n{}\n".format(vif_series1))

In [ ]:
col_list = ["screen_size", "weight","os_iOS"]

res = treating_multicollinearity(X_train, y_train, col_list)
res

**Because we do not concern ourselves with the elevated VIF for dummy variables, we are looking at the other elevated VIF values. Dropping os_iOS does not change the Adj R-squared so I will drop that column from the dataframe and recheck VIF to see if the regression VIF.**

In [ ]:
col_to_drop = "os_iOS"
X_train2 = X_train.loc[:, ~X_train.columns.str.startswith(col_to_drop)]
X_test2 = X_test.loc[:, ~X_test.columns.str.startswith(col_to_drop)]

In [ ]:
vif_series2 = pd.Series(
    [variance_inflation_factor(X_train2.values, i) for i in range(X_train2.shape[1])],
    index=X_train2.columns,
)
print("VIF values: \n\n{}\n".format(vif_series2))

**VIF are still high for screen size and weight but they affected the Adj R-squared so they will not be dropped.**

**Dropping high p-value variables**

- Dropping the predictor variables having a p-value greater than 0.05 is called for because they do not significantly impact the target variable.
- We will not drop them all at once so we can monitor the change in the p-values change after drop.
- To do this, we will build a loop that will check the p-values of the variables and drop the column with the highest p-value. Then it will create a new model without the dropped feature and check the p-values of the variables and drop the column with the highest p-value. The model will repeat these two steps till there are no columns with p-value > 0.05.


In [ ]:
# initial list of columns
cols = X_train2.columns.tolist()

# setting an initial max p-value
max_p_value = 1

while len(cols) > 0:
    # defining the train set
    x_train_aux = X_train2[cols]

    # fitting the model
    model = sm.OLS(y_train, x_train_aux).fit()

    # getting the p-values and the maximum p-value
    p_values = model.pvalues
    max_p_value = max(p_values)

    # name of the variable with maximum p-value
    feature_with_p_max = p_values.idxmax()

    if max_p_value > 0.05:
        cols.remove(feature_with_p_max)
    else:
        break

selected_features = cols
print(selected_features)

In [ ]:
X_train3 = X_train2[selected_features] 
X_test3 = X_test2[selected_features] 

In [ ]:
#rerunning the regression model after dropping all p-values greater than 1
olsmodel2 = sm.OLS(y_train,X_train3).fit() 
print(olsmodel2.summary())

**The Adj R-squared has not changed when the p-values above 1 were dropped. This tells us that those variables did not affect the models performance or that these variables significantly impact the target variable.**

**We will continue to check the other assumptions using this model**

TEST FOR LINEARITY AND INDEPENDENCE

In [ ]:
#creating a dataframe with actual, fitted and residual values
df_pred = pd.DataFrame()

df_pred["Actual Values"] = y_train  # actual values
df_pred["Fitted Values"] = olsmodel2.fittedvalues  # predicted values
df_pred["Residuals"] = olsmodel2.resid  # residuals

df_pred.head()

In [ ]:
 #plotting the fitted values vs residuals

sns.residplot(
    data=df_pred, x="Fitted Values", y="Residuals", color="purple", lowess=True
)
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.title("Fitted vs Residual plot")
plt.show()

**There is no real pattern to the fitted vs residual values. This supports the assumptions of linearity and independence.**

In [ ]:
# columns in training set
X_train3.columns

In [ ]:
# checking the distribution of variables in training set with dependent variable
sns.pairplot(df[["normalized_used_price",'screen_size', 'main_camera_mp', 'selfie_camera_mp', 'ram','battery', 'weight', 'normalized_new_price', 'years_since_release']])
plt.show()

**There appears to be a negative linear relationship between the normalized new price and normalized used price. We will transform this variable and rerun the regression.**

In [ ]:
# using square transformation
X_train3["normalized_new_price_sq"] = np.square(X_train3["normalized_new_price"])

# let's create a model with the transformed data
olsmodel3 = sm.OLS(y_train, X_train3)
olsmodel3 = olsmodel3.fit()
print(olsmodel3.summary())

In [ ]:
# let us recreate the dataframe with actual, fitted and residual values
df_pred = pd.DataFrame()

df_pred["Actual Values"] = y_train.values.flatten()  # actual values
df_pred["Fitted Values"] = olsmodel3.fittedvalues.values  # predicted values
df_pred["Residuals"] = olsmodel3.resid.values  # residuals

df_pred.head()

In [ ]:
# let us plot the fitted values vs residuals
sns.set_style("whitegrid")
sns.residplot(
    data=df_pred, x="Fitted Values", y="Residuals", color="purple", lowess=True
)
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.title("Fitted vs Residual plot")
plt.show()

**By transforming the normalized new price and rerunning the regression, there is still no linear relationship between the residuals and the Adjusted R-squared increased slightly.**

TEST FOR NORMALITY

In [ ]:
#Normality will be testing with the Q-Q plot of residuals and by using the Shapiro-Wilk test
sns.histplot(data=df_pred, x='Residuals', kde=True)
plt.title("Normality of eRsiduals")
plt.show()

**The residuals almost have a bell shape that is slightly skewed to the left.**


In [ ]:
#Checking the Q-Q plot for normal distribution of residuals
import pylab
import scipy.stats as stats

stats.probplot(df_pred["Residuals"], dist="norm", plot=pylab) ## Complete the code check Q-Q plot
plt.show()

**The residuals almost follow a straight line except for the tails.**

In [ ]:
#Checking the Shapiro-Wilks Test
#If the the p-value is greater than .05 the assumption is the that the residuals are normally distributed
stats.shapiro(df_pred["Residuals"])

**Because the Shapiro-Wilks p-value it is less than .05, we can say that the residuals are not normally distribution per this test. However, both the Q-Q and distribution plot indicate that the residuals are normally distributed. Per these results, the assumption is satisfied.**

TEST FOR HOMOSCEDASTICITY

In [ ]:
#Using the goldfeldquandt test, if we get a p-value greater than 0.05 the residuals are homoscedastic. Otherwise, they are heteroscedastic.
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

name = ["F statistic", "p-value"]
test = sms.het_goldfeldquandt(df_pred["Residuals"], X_train3)
lzip(name, test)

**Since the p-value for the goldfeldquandt test is greater than .05, we can show that the residuals are homoscedastic.**

**All of the assumptions of linear regression have been satified.**

## Final Model

In [ ]:
print(olsmodel3.summary())

In [ ]:
# let's check the model parameters
olsmodel3.params

In [ ]:
# Let us write the equation of linear regression
Equation = "mpg ="
print(Equation, end=" ")
for i in range(len(X_train3.columns)):
    if i == 0:
        print(olsmodel3.params[i], "+", end=" ")
    elif i != len(X_train3.columns) - 1:
        print(
            olsmodel3.params[i],
            "* (",
            X_train3.columns[i],
            ")",
            "+",
            end="  ",
        )
    else:
        print(olsmodel3.params[i], "* (", X_train3.columns[i], ")")

**MAKING PREDITIONS ON TEST DATA**

In [ ]:
#Checking the training columns in the final model
X_train3.columns

In [ ]:
#checking the test columns in the original data
X_test.columns

In [ ]:
# dropping columns from the test data that are not there in the training data
X_test2 = X_test.drop(
    ['int_memory','days_used','brand_name_Alcatel',
       'brand_name_Apple', 'brand_name_Asus', 'brand_name_BlackBerry',
       'brand_name_Celkon', 'brand_name_Coolpad', 'brand_name_Gionee',
       'brand_name_Google', 'brand_name_HTC', 'brand_name_Honor',
       'brand_name_Huawei', 'brand_name_Infinix', 'brand_name_Karbonn',
       'brand_name_LG', 'brand_name_Lava','brand_name_Meizu', 'brand_name_Micromax', 'brand_name_Microsoft',
       'brand_name_Motorola','brand_name_OnePlus',
       'brand_name_Oppo', 'brand_name_Others', 'brand_name_Panasonic',
       'brand_name_Realme', 'brand_name_Samsung', 'brand_name_Sony',
       'brand_name_Spice', 'brand_name_Vivo', 'brand_name_XOLO','brand_name_ZTE','os_Windows',
       'os_iOS','5g_yes' ], axis=1
)

In [ ]:
# transforming the weight column in the test data corresponding to the training set
X_test2["normalized_new_price_sq"] = np.square(X_test2["normalized_new_price"])

In [ ]:
# let's make predictions on the test set
y_pred = olsmodel3.predict(X_test2)

In [ ]:
# let's check the RMSE on the train data
rmse1 = np.sqrt(mean_squared_error(y_train, df_pred["Fitted Values"]))
rmse1

In [ ]:
# let's check the RMSE on the test data
rmse2 = np.sqrt(mean_squared_error(y_test, y_pred))
rmse2

**The RMSE on the train and test data is comparable. So the model is not overfitted.**

In [ ]:
# let's check the MAE on the train data
mae1 = mean_absolute_error(y_train, df_pred["Fitted Values"])
mae1

In [ ]:
# let's check the MAE on the test data
mae2 = mean_absolute_error(y_test, y_pred)
mae2

**The MAE on the training and test data indicate that predict the normalized used price within a mean error of .18 on the test data.**

**These factors make it seem that olsmodel3 is a good predictor of the factors contibuting to normalized used price.**

In [ ]:
#for use in insights
olsmodel3.params

## Actionable Insights and Recommendations 

-A unit increase in the screen size of a device will increase the normalized used device price by .021 Euros.
-A megapixal unit increase in the main camera will increase the normalized used device price by .017 Euros.
- A megapixal unit increase in the selfie camera will increase the normalized used device price by .014 Euros.
- A unit increase in RAM will increase the normalized used device price by .021 Euros.
-A unit increase in the longevity of the battery will decrease the normalized used device price by .000016 Euros, indicating that there is not a big difference in price due to battery size.
=The normalized new price of a comparable device has the biggest impact on the normalized used price. That amount is an increase of 1.04 Euros.
-For every year that the device is older, there is a decreased in the normalized used price of .027 Euros.
-Brand names do matter with Xiaomi being the most impactful with a .086 Euro increase in price. But there is an increase in price seen with Lenovo, Nokia and others as well.
There is an increase in the normalized used price when the phone has 4g capability of .034 Euros.

**The take aways from the data are:**
- ReCell should focus on name brand phones that have the following features:
    - A higher normalized new price for compariable devices.
    - Brand name recognition
    - 4g or higher capability
    - Higher RAM and screen size
    - Followed by the megapixal amount for both the main and selfie cameras.


In [ ]:
%%shell
jupyter nbconvert --to html /content/Apligian_ReCell_FullCode_Project_LearnerNotebook.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

___